In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from torchvision import models, transforms
from torch.utils.data import random_split
from tqdm import tqdm

In [2]:
# 图片预处理
class ImageTransform():

    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                # data augmentation
                transforms.RandomResizedCrop(
                   resize, scale=(0.5, 1.0)),
                transforms.RandomHorizontalFlip(), 
                # convert to tensor for PyTorch
                transforms.ToTensor(),
                # color normalization
                transforms.Normalize(mean, std)
            ]),
            'val': transforms.Compose([
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ])
        }

    def __call__(self, img, phase='train'):

        return self.data_transform[phase](img)
    
def read_file(file, pos=True):
    path = ""
    if pos:
        path = "/home/data/image_classifier/target/"
    else:
        path = "/home/data/image_classifier/no_target/"
    file_list = []
    for line in open(file):
        file_list.append(path + line.strip().split("/")[-1])

    return file_list

# 把传进来的file list切分为训练集和验证集
def split_train_and_val(path_file, train_ratio, pos=True):
        file_list = read_file(path_file, pos)
        num_samples = len(file_list)
        train_size = int(train_ratio * num_samples)
        val_size = num_samples - train_size
        train_file_list, val_file_list = random_split(file_list, [train_size, val_size])
        print(f"训练集大小:{len(train_file_list)},验证集大小:{len(val_file_list)}")
        return train_file_list, val_file_list

In [3]:
# 训练数据DataSet

class forestDataset(data.Dataset):

    def __init__(self, pos_file_list, neg_file_list, transform=None, phase='train'):
        self.pos_file_list = self.check_image_invalid(pos_file_list, True)
        self.neg_file_list = self.check_image_invalid(neg_file_list, False)
        self.total_file_list = self.pos_file_list + self.neg_file_list
        self.transform = transform
        self.phase = phase
    
    # 判断图片路径是否有效
    def check_image_invalid(self, file_list, pos=True):
        valid_image_list = []
        if pos:
            fd = open('invald_image_pos_data.txt', 'w')
        else:
            fd = open('invald_image_neg_data.txt', 'w')
        
        for file in file_list:
            try:
                Image.open(file)
                valid_image_list.append(file)
            except Exception as e:
                fd.write(file + "\n")

        print(f"invalid image size:{len(file_list) - len(valid_image_list)},input image size:{len(file_list)}")
        return valid_image_list

    def __len__(self):
        return len(self.total_file_list)

    def __getitem__(self, index):

        # load image
        img_path = self.total_file_list[index]

        # 读取到的图片有问题，则返回None
        img_originalsize = Image.open(img_path)

        # resize
        img = img_originalsize.resize((256, 256))

        # preprocess
        img_transformed = self.transform(
            img, self.phase)  # torch.Size([3, 224, 224])

        # picking up labels
        label = int(img_path in self.pos_file_list)


        return img_transformed, label
    

In [4]:
size = 256
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

train_ratio = 0.9

train_pos_file_list, val_pos_file_list = split_train_and_val(path_file="../have_target_images.txt", train_ratio=train_ratio, pos=True)
train_neg_file_list, val_neg_file_list = split_train_and_val(path_file="../no_target_images.txt", train_ratio=train_ratio, pos=False)

训练集大小:101660,验证集大小:11296
训练集大小:81944,验证集大小:9105


In [6]:
train_dataset = forestDataset(train_pos_file_list, train_neg_file_list, transform=ImageTransform(size, mean, std), phase='train')
val_dataset = forestDataset(val_pos_file_list, val_neg_file_list, transform=ImageTransform(size, mean, std), phase='val')

batch_size = 32

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=28, prefetch_factor=16, drop_last=True)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=28, prefetch_factor=16, drop_last=True)

# put dataloader into dictionary type
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

invalid image size:2,input image size:101660
invalid image size:17,input image size:81944
invalid image size:0,input image size:11296
invalid image size:3,input image size:9105


In [8]:
import hashlib
def get_file_path(pic_id):
    # 计算图片id的md5
    md5hash = hashlib.md5(pic_id.encode('utf-8'))
    md5 = md5hash.hexdigest()

    root_dir = "/hpc_input_fs"

    first = md5[0]
    second = md5[1:3]
    third = md5[3:6]
    path = "/".join([root_dir, first, second, third, pic_id])
    print(path)

### 使用vgg16预训练模型

In [9]:
# load pretrained vgg16 from PyTorch as an instance
# need to make setting 'internet' to 'On'.
use_pretrained = True
net = models.vgg16(pretrained=use_pretrained)

# Replace output layer for 2 class classifier, 
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)

net.train()

/root/miniconda3/envs/torch2/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/miniconda3/envs/torch2/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [10]:
# setting of loss function
criterion = nn.CrossEntropyLoss()

# setting fine tuned parameters

params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

# Not only output layer, "features" layers and other classifier layers are tuned.
update_param_names_1 = ["features"]
update_param_names_2 = ["classifier.0.weight",
                        "classifier.0.bias", "classifier.3.weight", "classifier.3.bias"]
update_param_names_3 = ["classifier.6.weight", "classifier.6.bias"]

# store parameters in list
for name, param in net.named_parameters():
    if update_param_names_1[0] in name:
        param.requires_grad = True
        params_to_update_1.append(param)
        #print("params_to_update_1:", name)

    elif name in update_param_names_2:
        param.requires_grad = True
        params_to_update_2.append(param)
        #print("params_to_update_2:", name)

    elif name in update_param_names_3:
        param.requires_grad = True
        params_to_update_3.append(param)
        #print("params_to_update_3:", name)

    else:
        param.requires_grad = False
        #print("no learning", name)

# print("-----------")
# print(params_to_update_1)

# Learning Rates
optimizer = optim.SGD([
    {'params': params_to_update_1, 'lr': 1e-4},
    {'params': params_to_update_2, 'lr': 5e-4},
    {'params': params_to_update_3, 'lr': 1e-3}
], momentum=0.9)

In [11]:
# training function
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    
    accuracy_list = []
    loss_list = []
    
    # Precondition : Accelerator GPU -> 'On'
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("using device：", device)

    # put network into GPU
    net.to(device)
    torch.backends.cudnn.benchmark = True

    # epoch loop
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # set network 'train' mode
            else:
                net.eval()   # set network 'val' mode

            epoch_loss = 0.0
            epoch_corrects = 0

            # Before training
            if (epoch == 0) and (phase == 'train'):
                continue
            
            # batch loop
            for inputs, labels in tqdm(dataloaders_dict[phase]):
                # send data to GPU
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # initialize optimizer
                optimizer.zero_grad()

                # forward
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)

                    loss = criterion(outputs, labels)  #calcurate loss
                    _, preds = torch.max(outputs, 1)  # predict
  
                    # back propagtion
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # update loss summation
                    epoch_loss += loss.item() * inputs.size(0)  
                    # update correct prediction summation
                    epoch_corrects += torch.sum(preds == labels.data)

            # loss and accuracy for each epoch loop
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
            if phase == 'val':
                accuracy_list.append(epoch_acc.item())
                loss_list.append(epoch_loss)
            
    return accuracy_list, loss_list

In [12]:
num_epochs=10
accuracy_list, loss_list = train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

torch.save(net.state_dict, "image_classifier_model_v1.pth")

using device： cuda:0
Epoch 1/10
-------------


100%|██████████| 637/637 [02:49<00:00,  3.75it/s]


val Loss: 0.7525 Acc: 0.4353
Epoch 2/10
-------------


100%|██████████| 5737/5737 [54:44<00:00,  1.75it/s]


train Loss: 0.2925 Acc: 0.8665


...

100%|██████████| 5737/5737 [54:28<00:00,  1.76it/s]


train Loss: 0.2379 Acc: 0.8950


100%|██████████| 637/637 [02:28<00:00,  4.29it/s]


val Loss: 0.2066 Acc: 0.9112
Epoch 4/10
-------------


 89%|████████▉ | 5100/5737 [48:25<06:00,  1.77it/s]